In [2]:
import pandas as pd
from collections import defaultdict

In [3]:
import random
import numpy as np

# Load the clean dataset
clean_data = pd.read_csv('original_dataset.csv')

# Create a copy of the clean dataset
dirty_data = clean_data.copy()

# Introduce missing values
columns_to_modify = ['Depression', 'BMI', 'Alcohol_Consumption']
for column in columns_to_modify:
    dirty_data[column] = dirty_data[column].astype(str)
    num_missing = int(len(dirty_data) * 0.1)  # Introduce 10% missing values
    missing_indices = random.sample(range(len(dirty_data)), num_missing)
    dirty_data.loc[missing_indices, column] = 'Nan'

# Introduce typos and inconsistent values
columns_to_modify = ['Sex', 'Age_Category', 'Checkup']
for column in columns_to_modify:
    num_errors = int(len(dirty_data) * 0.1)  # Introduce errors in 5% of the rows
    error_indices = random.sample(range(len(dirty_data)), num_errors)
    for index in error_indices:
        if column == 'Sex':
            dirty_data.at[index, column] = 'Femle' if dirty_data.at[index, column] == 'Female' else 'ale'
        elif column == 'Age_Category':
            dirty_data.at[index, column] = dirty_data.at[index, column].replace('-', '')
        elif column == 'Checkup':
            dirty_data.at[index, column] = dirty_data.at[index, column].replace('W', 'w')

# Introduce invalid values
columns_to_modify = ['Height_(cm)', 'Weight_(kg)', 'FriedPotato_Consumption']
for column in columns_to_modify:
    num_invalid = int(len(dirty_data) * 0.05)  # Introduce invalid values in 2% of the rows
    invalid_indices = random.sample(range(len(dirty_data)), num_invalid)
    for index in invalid_indices:
        if column == 'Height_(cm)':
            dirty_data.at[index, column] = random.choice([0, 1000])
        elif column == 'Weight_(kg)':
            dirty_data.at[index, column] = random.choice([0, 500])
        elif column == 'FriedPotato_Consumption':
            dirty_data.at[index, column] = random.choice([-10, 100])

# Save the dirty dataset
dirty_data.to_csv('dirty_dataset.csv', index=False)

In [4]:
data = pd.read_csv('dirty_dataset.csv')
df = pd.DataFrame(data)

In [5]:
df['General_Health'].unique()

# for col in df.columns:
#     print(col, df[col].unique().shape)

array(['Poor', 'Very Good', 'Good', 'Fair', 'Excellent'], dtype=object)

In [6]:
# Define rules for abnormality for each column
def age_rule1(x):
    return not '-' in x and (x != '80+')

def starts_with_W_rule(x):
    return x.startswith('w')

def sex_rule(x):
    return x not in ['Male', 'Female']


def depression_rule(x):
        return x == 'Nan'  
    
rules = {
    'FriedPotato_Consumption': lambda x: x < 0, 
    'Depression': depression_rule,
    'BMI': depression_rule,
    'Alcohol_Consumption': depression_rule,
    'Height_(cm)': lambda x: (x < 50) | (x > 250),
    'Weight_(kg)': lambda x: (x < 30) | (x > 200),
    'Sex': sex_rule,
    'Checkup': starts_with_W_rule,
    'Age_Category': age_rule1,
}

In [7]:
# Initialize dictionaries to store indices of rows for each group
groups = {column: [] for column in ['Checkup', 'Depression', 'Sex',
       'Age_Category', 'Height_(cm)', 'Weight_(kg)', 'BMI',
       'Alcohol_Consumption', 'FriedPotato_Consumption']}

# Iterate over each row and assign it to the appropriate group
for index, row in df.iterrows():
    for column, rule in rules.items():
        if rule(row[column]):
            groups[column].append(index)
            break

# Create DataFrames for each group
abnormal_group_data = {}
cleaned_group_data = {}
for column, indices in groups.items():
    abnormal_group_data[column] = df.loc[indices]
    cleaned_group_data[column] = df.drop(indices)

for column, data in abnormal_group_data.items():
    data.to_csv(f'{column}_abnormal_group.csv', index=False)

for column, data in cleaned_group_data.items():
    data.to_csv(f'{column}_cleaned_group.csv', index=False)    

In [8]:
cleaned_group_data

{'Checkup':        General_Health                  Checkup Exercise Heart_Disease  \
 0                Poor  Within the past 2 years       No            No   
 1           Very Good     Within the past year       No           Yes   
 2           Very Good     Within the past year      Yes            No   
 3                Poor     Within the past year      Yes           Yes   
 4                Good     Within the past year       No            No   
 ...               ...                      ...      ...           ...   
 308849      Very Good     Within the past year      Yes            No   
 308850           Fair  Within the past 5 years      Yes            No   
 308851      Very Good      5 or more years ago      Yes            No   
 308852      Very Good     Within the past year      Yes            No   
 308853      Excellent     Within the past year      Yes            No   
 
        Skin_Cancer Other_Cancer Depression  \
 0               No           No         No   
 1   

In [9]:
pd.read_csv('FriedPotato_Consumption_abnormal_group.csv')
# pd.read_csv('Depression_abnormal_group.csv')
# pd.read_csv('BMI_abnormal_group.csv')
# pd.read_csv('Alcohol_Consumption_abnormal_group.csv')
# pd.read_csv('Height_(cm)_abnormal_group.csv')
# pd.read_csv('Weight_(kg)_abnormal_group.csv')
# pd.read_csv('Sex_abnormal_group.csv')
# pd.read_csv('Checkup_abnormal_group.csv')
# pd.read_csv('Age_Category_abnormal_group.csv')

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,Excellent,Within the past year,Yes,No,No,No,No,No,Yes,Male,7074,191.0,112.49,Nan,No,0.0,30.0,10.0,-10.0
1,Fair,Within the past year,Yes,No,No,No,Yes,No,Yes,Female,60-64,168.0,85.28,30.34,Yes,4.0,0.0,3.0,-10.0
2,Good,Within the past year,Yes,No,No,No,Yes,No,No,Female,50-54,157.0,61.23,Nan,Yes,0.0,8.0,25.0,-10.0
3,Good,Within the past year,Yes,No,No,No,Yes,Yes,Yes,Female,55-59,160.0,77.56,30.29,No,0.0,60.0,30.0,-10.0
4,Fair,Within the past year,No,No,No,No,Nan,No,No,Male,60-64,175.0,67.13,21.86,No,0.0,30.0,0.0,-10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7642,Very Good,Within the past 2 years,Yes,No,No,No,No,No,No,Male,60-64,178.0,80.74,25.54,No,5.0,16.0,16.0,-10.0
7643,Good,5 or more years ago,Yes,No,No,No,No,No,No,Male,60-64,175.0,86.18,28.06,Yes,Nan,16.0,8.0,-10.0
7644,Very Good,Within the past year,Yes,No,No,Yes,No,No,Yes,Female,60-64,183.0,0.00,18.99,No,20.0,120.0,60.0,-10.0
7645,Good,Within the past year,Yes,No,No,No,Yes,No,No,Male,30-34,175.0,500.00,22.15,Yes,0.0,12.0,8.0,-10.0


In [10]:
pd.read_csv('FriedPotato_Consumption_cleaned_group.csv')
# pd.read_csv('Depression_cleaned_group.csv')
# pd.read_csv('BMI_cleaned_group.csv')
# pd.read_csv('Alcohol_Consumption_cleaned_group.csv')
# pd.read_csv('Height_(cm)_cleaned_group.csv')
# pd.read_csv('Weight_(kg)_cleaned_group.csv')
# pd.read_csv('Sex_cleaned_group.csv')
# pd.read_csv('Checkup_cleaned_group.csv')
# pd.read_csv('Age_Category_cleaned_group.csv')

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,Poor,Within the past 2 years,No,No,No,No,No,No,Yes,Femle,70-74,150.0,32.66,14.54,Yes,0.0,30.0,16.0,12.0
1,Very Good,Within the past year,No,Yes,No,No,No,Yes,No,Female,70-74,165.0,77.11,28.29,No,0.0,30.0,0.0,4.0
2,Very Good,Within the past year,Yes,No,No,No,No,Yes,No,Female,60-64,163.0,88.45,33.47,No,4.0,12.0,3.0,16.0
3,Poor,Within the past year,Yes,Yes,No,No,No,Yes,No,Male,7579,180.0,93.44,28.73,No,0.0,30.0,30.0,8.0
4,Good,Within the past year,No,No,No,No,No,No,No,Male,80+,191.0,88.45,24.37,Yes,0.0,8.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301202,Very Good,Within the past year,Yes,No,No,No,No,No,No,ale,25-29,168.0,81.65,29.05,No,4.0,30.0,8.0,0.0
301203,Fair,Within the past 5 years,Yes,No,No,No,No,Yes,No,ale,6569,180.0,69.85,21.48,No,8.0,15.0,60.0,4.0
301204,Very Good,5 or more years ago,Yes,No,No,No,Yes,"Yes, but female told only during pregnancy",No,Female,30-34,157.0,61.23,24.69,Yes,4.0,40.0,8.0,4.0
301205,Very Good,Within the past year,Yes,No,No,No,Nan,No,No,Male,65-69,183.0,79.38,23.73,No,3.0,30.0,12.0,0.0


In [19]:
import pandas as pd

# The columns to handle
columns = [
    'FriedPotato_Consumption', 'Depression', 'BMI', 
    'Alcohol_Consumption', 'Height_(cm)', 'Weight_(kg)', 
    'Sex', 'Checkup', 'Age_Category'
]

# Categorical and numerical columns
categorical_columns = ['General_Health', 'Checkup', 'Exercise', 'Heart_Disease', 
                       'Skin_Cancer', 'Other_Cancer', 'Depression', 'Diabetes', 
                       'Arthritis', 'Sex', 'Age_Category', 'Smoking_History']
numerical_columns = ['Height_(cm)', 'Weight_(kg)', 'BMI', 'Alcohol_Consumption', 
                     'Fruit_Consumption', 'Green_Vegetables_Consumption', 'FriedPotato_Consumption']

# Similarity score function
def calculate_similarity_score(row1, row2):
    score = 0
    score += sum(row1[col] == row2[col] for col in categorical_columns)
    score += sum(abs(row1[col] - row2[col]) <= (cleaned_df[col].std()) for col in numerical_columns if col in row2 and not pd.isnull(row1[col]) and not pd.isnull(row2[col]))
    return score

# Process each column's datasets
for col in columns:
    # Load datasets
    abnormal_df = pd.read_csv(f'{col}_abnormal_group.csv')
    cleaned_df = pd.read_csv(f'{col}_cleaned_group.csv')

    # Convert numeric columns to floats and handle missing values
    for column in numerical_columns:
        if column in cleaned_df.columns:
            cleaned_df[column] = pd.to_numeric(cleaned_df[column], errors='coerce')
            abnormal_df[column] = pd.to_numeric(abnormal_df[column], errors='coerce')

    # Find and replace abnormal data
    for index, abnormal_row in abnormal_df.iterrows():
        max_score = -1
        most_similar_row = None
        for _, clean_row in cleaned_df.iloc[:2].iterrows():
            similarity_score = calculate_similarity_score(abnormal_row, clean_row)
            if similarity_score > max_score:
                max_score = similarity_score
                most_similar_row = clean_row

        if most_similar_row is not None:
            abnormal_df.at[index, col] = most_similar_row[col]

    # Save the updated abnormal group dataset
    abnormal_df.to_csv(f'{col}_abnormal_group_updated.csv', index=False)


In [1]:
import pandas as pd

# Define the columns to handle
columns = [
    'FriedPotato_Consumption', 'Depression', 'BMI', 
    'Alcohol_Consumption', 'Height_(cm)', 'Weight_(kg)', 
    'Sex', 'Checkup', 'Age_Category'
]

# Initialize an empty dataframe to hold all concatenated data
combined_df = pd.DataFrame()

for col in columns:
    # Load the updated abnormal data and the clean data
    updated_abnormal_df = pd.read_csv(f'{col}_abnormal_group_updated.csv')
    clean_df = pd.read_csv(f'{col}_cleaned_group.csv')

    # Concatenate the updated abnormal data with the clean data
    full_data_df = pd.concat([updated_abnormal_df, clean_df], ignore_index=True)

    # If combined_df is empty, initialize it with full_data_df
    if combined_df.empty:
        combined_df = full_data_df.copy()
    else:
        # Ensure that the combined dataframe only has unique rows
        combined_df = pd.concat([combined_df, full_data_df]).drop_duplicates().reset_index(drop=True)


combined_df.head()
combined_df.shape

# Save the final combined dataframe
combined_df.to_csv('final_combined_dataset.csv', index=False)

In [2]:
import pandas as pd

# Load the datasets
original_df = pd.read_csv('original_dataset.csv')

# Convert all columns to the same type for a fair comparison
for col in combined_df.columns:
    if combined_df[col].dtype == 'float64' or original_df[col].dtype == 'float64':
        combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')
        original_df[col] = pd.to_numeric(original_df[col], errors='coerce')
    else:
        combined_df[col] = combined_df[col].astype(str)
        original_df[col] = original_df[col].astype(str)

# Now attempt the merge
exact_matches = combined_df.merge(original_df, indicator=True, how='inner').query('_merge == "both"')
exact_match_rate = len(exact_matches) / len(original_df)

# Output the match rate
print(f'Exact match rate: {exact_match_rate:.2%}')


Exact match rate: 58.49%


Note on Handling Multiple Abnormalities within Single Records:


In our current approach to cleaning data with the Integrated Healthcare Cleaning System (IHCS), we process abnormalities grouped by type, focusing on correcting each type effectively. However, this method overlooked the possibility that a single record might contain multiple types of errors, each falling into different abnormality groups.

The issue arises when such a multi-error record is processed separately for each error type. For instance, if a record has both abnormal BMI and Alcohol Consumption values, it is corrected once for BMI under the BMI group and again for Alcohol Consumption under its respective group. Consequently, when these records are merged back into the main dataset, the same record can appear twice—each time corrected for different abnormalities but still containing other uncorrected errors.

To resolve this, a more effective strategy would involve assigning a unique index to each record at the outset of the cleaning process. This index would be used to update the record directly in the dataset each time an error is corrected, regardless of the error group. This approach ensures that each record is amended in-place, maintaining its integrity and preventing the duplication of records in the final cleaned dataset. This change will significantly enhance the accuracy and efficiency of the data cleaning process by ensuring that corrections across multiple groups are consolidated into a single, coherent record update. We regret that this improvement was recognized too late for inclusion in the current dataset and ask for understanding regarding this limitation in our evaluation.